In [1]:
using CSV
using DataFrames

In [2]:
using JuMP
using Cbc

## Exploring the Data
First, let's load our 2018-2019 player data. This CSV file contains statistics for active US-born players in the 2018-2019 season. We have also added in information about whether each player appeared in the previous Olympics (2016) and the most recent NBA All-Star game as additional measures. 

Source: https://www.basketball-reference.com/

In [3]:
df = CSV.read("NBA_data_2018_2019.csv");
names(df)

32-element Array{Symbol,1}:
 :ID              
 :Player          
 :Username        
 :Pos             
 :Age             
 :Tm              
 :G               
 :MP              
 :PER             
 Symbol("TS.")    
 :X3PAr           
 :FTr             
 Symbol("ORB.")   
 ⋮                
 :OWS             
 :DWS             
 :WS              
 :WS_48           
 :OBPM            
 :DBPM            
 :BPM             
 :VORP            
 :Olympics2016    
 :OlympicsFinalist
 :AllStar         
 :HighRated       

Let's see what this data looks like:

In [4]:
first(df,10)

,ID,Player,Username,Pos,Age,Tm,G,MP,PER
,Int64,String,String,String,Int64,String,Int64,Int64,Float64
1,5,Bam Adebayo,adebaba01,C,21,MIA,82,1913,17.9
2,8,LaMarcus Aldridge,aldrila01,C,33,SAS,81,2687,22.9
3,11,Jarrett Allen,allenja01,C,20,BRK,80,2096,18.5
4,13,Al-Farouq Aminu,aminual01,PF,28,POR,81,2292,13.2
5,20,Carmelo Anthony,anthoca01,PF,34,HOU,10,294,10.9
6,22,Ryan Arcidiacono,arcidry01,PG,24,CHI,81,1961,11.6
7,23,Trevor Ariza,arizatr01,SF,33,TOT,69,2349,12.0
8,24,D.J. Augustin,augusdj01,PG,31,ORL,81,2269,15.7
9,27,Marvin Bagley,baglema01,PF,19,SAC,62,1567,18.9


## A Simple Model

We will build our optimization models using the JuMP package in Julia: http://www.juliaopt.org/JuMP.jl/v0.19.0/

First, we have to initialize a model in JuMP using the Model() function. We also tell Julia that we are using the **Cbc** solver. This is an open-source solver that allows us to solve integer optimization problems. Other popular alternatives are **Gurobi** and **CPLEX**, but these require licenses. 

In [5]:
model = Model(Cbc.Optimizer)

A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: COIN Branch-and-Cut (Cbc)

### I. Defining the Decision Variables
First we need to create our decision variables. Remember, we want to construct variables:

$$
    x_i=\left\{
                \begin{array}{ll}
                  1 & \text{if player $i$ is selected for the team,}\\
                  0 & \text{otherwise.}
                \end{array}
              \right.
$$

These are called **binary** variables since they only take on values between 0 and 1. We need an $x_i$ for all $i=1,\ldots,N$.

In [7]:
N = nrow(df)
@variable(model, x[1:N], Bin)

200-element Array{VariableRef,1}:
 x[1]  
 x[2]  
 x[3]  
 x[4]  
 x[5]  
 x[6]  
 x[7]  
 x[8]  
 x[9]  
 x[10] 
 x[11] 
 x[12] 
 x[13] 
 ⋮     
 x[189]
 x[190]
 x[191]
 x[192]
 x[193]
 x[194]
 x[195]
 x[196]
 x[197]
 x[198]
 x[199]
 x[200]

### II. Formalizing the Objective

We'll start with a basic objective, which is simply to maximize the average Player Efficiency Rating (PER) of the players on the selected team. We'll denote player $i$'s PER as $c_i$. We can then calculate the objective as:

$$ \frac{1}{12} \sum_{i=1}^{N}(x_i*s_i)$$

We can easily formulate this in JuMP:

In [8]:
s = df[!,:PER]
@objective(model, Max, 1/12*sum(x[i]*s[i] for i=1:N));

### III. Constructing the Constraints 

Now that we have defined our variables and quantified our goal, we need to specify what requirements any team must satisfy. Let's start by just placing a constraint on team size. 

In [9]:
@constraint(model, sum(x[i] for i=1:N) == 12);

### IV. Solving the Model
We have specified the three key elements of any mixed-integer optimization model: 
- Decision Variables
- Objective 
- Feasibility Constraints

We're ready to solve the model!

In [10]:
model

A JuMP Model
Maximization problem with:
Variables: 200
Objective function type: GenericAffExpr{Float64,VariableRef}
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.EqualTo{Float64}`: 1 constraint
`VariableRef`-in-`MathOptInterface.ZeroOne`: 200 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: COIN Branch-and-Cut (Cbc)
Names registered in the model: x

In [11]:
optimize!(model);

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Oct  7 2019 

command line - Cbc_C_Interface -solve -quit (default strategy 1)
Continuous objective value is 25.375 - 0.00 seconds
Cgl0004I processed model has 1 rows, 200 columns (200 integer (200 of which binary)) and 200 elements
Cbc0012I Integer solution of -25.375 found by DiveCoefficient after 0 iterations and 0 nodes (0.00 seconds)
Cbc0001I Search completed - best objective -25.375, took 0 iterations and 0 nodes (0.00 seconds)
Cbc0035I Maximum depth 0, 0 variables fixed on reduced cost
Cuts at root node changed objective from -25.375 to -25.375
Probing was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
Gomory was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
Knapsack was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
Clique was tried 0 times and created 0 cuts o

Let's see what our results look like: we can look at the values of our decision variables $x_1,\ldots,x_N$ to see which indices were set to 1. These indices correspond to the players that were selected for the team.

In [12]:
selection_indices = value.(x);

In [14]:
selected_players = df[selection_indices.==1,:Player]

12-element Array{String,1}:
 "Stephen Curry"     
 "Anthony Davis"     
 "Andre Drummond"    
 "Kevin Durant"      
 "Paul George"       
 "James Harden"      
 "Montrezl Harrell"  
 "LeBron James"      
 "Kawhi Leonard"     
 "Damian Lillard"    
 "Karl-Anthony Towns"
 "Hassan Whiteside"  

In [15]:
objective_value(model)

25.374999999999996

How well does this match the list of finalists for the Olympics? 

In [16]:
df[selection_indices.==1,[:Player,:Pos,:AllStar,:Olympics2016,:OlympicsFinalist]]

,Player,Pos,AllStar,Olympics2016,OlympicsFinalist
,String,String,Int64,Int64,Int64
1,Stephen Curry,PG,1,0,1
2,Anthony Davis,C,1,0,1
3,Andre Drummond,C,0,0,1
4,Kevin Durant,SF,1,1,1
5,Paul George,SF,1,1,1
6,James Harden,PG,1,0,1
7,Montrezl Harrell,C,0,0,1
8,LeBron James,SF,1,0,1
9,Kawhi Leonard,SF,1,0,1


## Making a More Realistic Team

While this team is technically legal under Olympic regulations, it's not enough for us just to look at PER! We want to make sure that we have enough players of each position to fill out the team. We can add a few constraints to make a more useful team. 

In [17]:
## Our original model
model2 = Model(Cbc.Optimizer)
@variable(model2, x[1:N], Bin)
@objective(model2, Max, 1/12*sum(x[i]*df[i,:PER] for i=1:N));
@constraint(model2, sum(x[i] for i=1:N) == 12);

#### Position Requirements

Suppose we want at least 4 guards, 4 forwards, and 3 centers. We first define indicators for each player's position. For example, to indicate which players are forwards, we define:


$$
    f_i=\left\{
                \begin{array}{ll}
                  1 & \text{if player $i$ is a forward,}\\
                  0 & \text{otherwise.}
                \end{array}
              \right.
$$

We can create similar variables $c_i$ and $g_i$ for center and guard positions. Note that this is **data**; these are not decision variables.

In [18]:
c = ifelse.(df[!,:Pos].=="C",1,0);
f = ifelse.((df[!,:Pos].=="PF").|(df[!,:Pos].=="SF"),1,0);
g = ifelse.((df[!,:Pos].=="PG").|(df[!,:Pos].=="SG"),1,0);

println("Center Count: ", sum(c))
println("Forward Count: ", sum(f))
println("Guard Count: ", sum(g))

Center Count: 33
Forward Count: 67
Guard Count: 96


We can create constraints by summing the position indicator over the players who are selected for the team, like we did with the objective:

$$ \sum_{i=1}^{N}(f_i*x_i) \geq 4$$

We can do the same for the center and guard positions as well.

In [19]:
# Forward constraint
@constraint(model2, sum(f[i]*x[i] for i=1:N) >= 4);

# Center constraint
@constraint(model2, sum(c[i]*x[i] for i=1:N) >= 3);

# Guard constraint
@constraint(model2, sum(g[i]*x[i] for i=1:N) >= 4);

#### Defensive Ability
We want to make sure that our team has good average defensive ability. One (imperfect!) way to measure this is using Defensive Box Plus/Minus (DBPM). We can require that the average DBPM of the chosen team is at least +1. 


$$ \frac{1}{12} \sum_{i=1}^{N}(d_i*x_i) \geq 1$$

where $d_i$ is player $i$'s DBPM. 

In [20]:
d = df[!,:DBPM]
@constraint(model2, 1/12 * sum(d[i]*x[i] for i=1:N) >= 1.0);

#### Experience Constraints
We also want to make sure our team has enough Olympics experience: let's add a constraint that at least 3 selected players were on the 2016 Olympic team.

We use a similar syntax as before:

$$ \sum_{i=1}^{N}(o_i*x_i) \geq 3 $$

where $o_i = 1$ if player $i$ was on the 2016 Olympic team, and 0 otherwise. 


In [21]:
o = df[!,:Olympics2016]
@constraint(model2, sum(o[i]*x[i] for i=1:N) >= 3);

Let's solve our new model.

In [22]:
set_silent(model2)
optimize!(model2)

In [25]:
selected_players2 = sort(df[value.(x).==1,:Player])

12-element Array{String,1}:
 "Andre Drummond"    
 "Anthony Davis"     
 "Damian Lillard"    
 "Hassan Whiteside"  
 "James Harden"      
 "Jimmy Butler"      
 "Karl-Anthony Towns"
 "Kawhi Leonard"     
 "Kevin Durant"      
 "LeBron James"      
 "Paul George"       
 "Stephen Curry"     

If we compare to the original player list, we see that we have removed Montrezl Harrell (Center) and added Jimmy Butler (Forward).

In [26]:
sort(selected_players)

12-element Array{String,1}:
 "Andre Drummond"    
 "Anthony Davis"     
 "Damian Lillard"    
 "Hassan Whiteside"  
 "James Harden"      
 "Karl-Anthony Towns"
 "Kawhi Leonard"     
 "Kevin Durant"      
 "LeBron James"      
 "Montrezl Harrell"  
 "Paul George"       
 "Stephen Curry"     

How good is our new model, according to our **objective function**?

In [27]:
objective_value(model2)

25.28333333333333

In [28]:
objective_value(model)

25.374999999999996

This is < 0.1 worse than our original solution: even with these new constraints, we haven't lost much.

## Variations of Our Basic Model


Before we experiment further, we can put our full model into a **function** that will let us easily modify the model for different objectives and datasets.

In [29]:
function create_roster(df, objective_metric)
    m = Model(Cbc.Optimizer)
    set_silent(m)
    
    # Define variables
    N = nrow(df)
    @variable(m, x[1:N], Bin)
    
    # Define relevant data columns
    c = ifelse.(df[!,:Pos].=="C",1,0);
    f = ifelse.((df[!,:Pos].=="PF").|(df[!,:Pos].=="SF"),1,0);
    g = ifelse.((df[!,:Pos].=="PG").|(df[!,:Pos].=="SG"),1,0);
    d = df[!,:DBPM];
    o = df[!,:Olympics2016];
    
    # Pull in data for our chosen objective
    s = df[!,objective_metric];
    
    # Objective
    @objective(m, Max, 1/12 * sum(x[i]*s[i] for i=1:N))
    
    # Team size constraint
    @constraint(m, sum(x[i] for i=1:N) == 12);
    
    # Position constraints
    @constraint(m, sum(f[i]*x[i] for i=1:N) >= 4);
    @constraint(m, sum(c[i]*x[i] for i=1:N) >= 3);
    @constraint(m, sum(g[i]*x[i] for i=1:N) >= 4);

    # Experience constraint
    @constraint(m, sum(o[i]*x[i] for i=1:N) >= 3);

    # Defensive ability constraint
    @constraint(m, 1/12 * sum(d[i]*x[i] for i=1:N) >= 1);
    
    optimize!(m)
    
    players = sort(df[value.(x).>0.99,:Player])
    return(objective_value(m), players)
end

create_roster (generic function with 1 method)

Now, it is easy for us to build models for different objective functions. We simply have to specify our dataset of interest and the column of the metric that we want to maximize.

#### Alternative Objective Functions

We chose to optimize average PER, but we could have chosen other metrics too. How does our team composition change when we use other measures of player performance? 

In [30]:
obj_per, players_per = create_roster(df,:PER)

(25.28333333333333, ["Andre Drummond", "Anthony Davis", "Damian Lillard", "Hassan Whiteside", "James Harden", "Jimmy Butler", "Karl-Anthony Towns", "Kawhi Leonard", "Kevin Durant", "LeBron James", "Paul George", "Stephen Curry"])

In [31]:
obj_bpm, players_bpm = create_roster(df,:BPM)

(6.541666666666666, ["Anthony Davis", "Damian Lillard", "James Harden", "Jimmy Butler", "Karl-Anthony Towns", "Kawhi Leonard", "Kevin Durant", "LeBron James", "Mitchell Robinson", "Paul George", "Russell Westbrook", "Stephen Curry"])

In [33]:
obj_ws48, players_ws48 = create_roster(df,:WS_48)

(0.2099166666666667, ["Anthony Davis", "Damian Lillard", "Derrick Favors", "James Harden", "Jimmy Butler", "Kawhi Leonard", "Kevin Durant", "LeBron James", "Mitchell Robinson", "Nerlens Noel", "Paul George", "Stephen Curry"])

#### How different is the playoff perspective?  

Now let's repeat the model (with the original PER objective), but using data from the past two playoff seasons instead. We're looking at average playoff statistics for all active US-born NBA players.

In [34]:
df_playoffs = CSV.read("NBA_data_playoffs_2017_2019.csv");
first(df_playoffs,10)

,ID,Player,Username,From,To,Tm,Lg,WS,G
,Int64,String,String,Int64,Int64,String,String,Float64,Int64
1,1,LeBron James,jamesle01,2017,2018,CLE,NBA,9.4,40
2,2,Kevin Durant,duranke01,2017,2019,GSW,NBA,9.0,48
3,3,Stephen Curry,curryst01,2017,2019,GSW,NBA,8.8,54
4,4,Kawhi Leonard,leonaka01,2017,2019,TOT,NBA,7.7,36
5,5,Draymond Green,greendr01,2017,2019,GSW,NBA,7.2,60
6,6,James Harden,hardeja01,2017,2019,HOU,NBA,5.1,39
7,7,Kyle Lowry,lowryky01,2017,2019,TOR,NBA,4.7,42
8,8,Chris Paul,paulch01,2017,2019,TOT,NBA,4.5,33
9,9,Andre Iguodala,iguodan01,2017,2019,GSW,NBA,3.8,52


We can leverage the same function, but now we want to speficy a new dataset, **df_playoffs**, rather than the original dataset **df**.

In [35]:
obj_playoff, players_playoff = create_roster(df_playoffs,:PER)

(30.866666666666667, ["Anthony Davis", "Chris Paul", "Draymond Green", "JaVale McGee", "James Harden", "Jimmy Butler", "Kawhi Leonard", "Kevin Durant", "Larry Nance", "LeBron James", "Malcolm Delaney", "Stephen Curry"])

We can create a table to compare how much these different lists overlap. We'll do this using the `join` function, which is similar to SQL syntax.

In [36]:
player_compare = join(
    DataFrame(Player = players_per, PER = 1), 
    DataFrame(Player = players_bpm, BPM = 1),
    DataFrame(Player = players_ws48, WS_48 = 1), 
    DataFrame(Player = players_playoff, PER_PLAYOFF = 1),
    kind = :outer, on = :Player)

sort(player_compare)

,Player,PER,BPM,WS_48,PER_PLAYOFF
,String,Int64⍰,Int64⍰,Int64⍰,Int64⍰
1,Andre Drummond,1,missing,missing,missing
2,Anthony Davis,1,1,1,1
3,Chris Paul,missing,missing,missing,1
4,Damian Lillard,1,1,1,missing
5,Derrick Favors,missing,missing,1,missing
6,Draymond Green,missing,missing,missing,1
7,Hassan Whiteside,1,missing,missing,missing
8,JaVale McGee,missing,missing,missing,1
9,James Harden,1,1,1,1


Finally, we can also add a column comparing our lists to the true list of Olympic Finalists for 2020.   

In [37]:
# Pull a dataset of the olympic finalists
olympics_finalists = unique(df[df[!,:OlympicsFinalist].==1,[:Player,:OlympicsFinalist]])

sort(join(player_compare, olympics_finalists, on = :Player, kind = :left))

,Player,PER,BPM,WS_48,PER_PLAYOFF,OlympicsFinalist
,String,Int64⍰,Int64⍰,Int64⍰,Int64⍰,Int64⍰
1,Andre Drummond,1,missing,missing,missing,1
2,Anthony Davis,1,1,1,1,1
3,Chris Paul,missing,missing,missing,1,1
4,Damian Lillard,1,1,1,missing,1
5,Derrick Favors,missing,missing,1,missing,missing
6,Draymond Green,missing,missing,missing,1,1
7,Hassan Whiteside,1,missing,missing,missing,missing
8,JaVale McGee,missing,missing,missing,1,1
9,James Harden,1,1,1,1,1


7 players (Anthony Davis, James Harden, Jimmy Butler, Kawhi Leonard, Kevin Durant, Lebron James, and Stephen Curry) are chosen in every variation of our model. Certain players only appear when we look at playoff statistics (e.g. Chris Paul, Draymond Green), while others look better when we focus on the regular season (e.g. Paul George, Damian Lillard).